# Retrive NFTs Assets by Address using CardanoScan API

---
### 1.1 **1.4 Necessary Package Installations and Open Statements**

In [ ]:
#r "nuget: FSharp.Data"

In [47]:
open System
open System.IO
open System.Text.Json
open System.Net.Http
open System.Threading.Tasks
open FSharp.Data

## Validate Your API Key

To use the [CardanoScan API](https://docs.cardanoscan.io/), you must have a valid API key. The `getApiKey` function retrieves this key from your environment variables, so ensure your key is named `CARDANO_API_KEY` to be recognized. 

1. **`getApiKey`**:
   - Retrieves your API key from the environment.
   - Ensures the key is securely accessed and avoids hardcoding sensitive information.

2. **`saveJsonToFile`**:
   - Formats the JSON response into a more readable format and saves it to a file.
   - By default, it saves the output to [`collectionassets.json`](./data/collectionassets.json).

3. **`getAssetsByAddress`**:
   - Configures the API request by setting the required headers (including the API key) and query parameters.
   - Sends the request to the API endpoint and handles the response.


In [26]:
let getApiKey () =
    match Environment.GetEnvironmentVariable("CARDANO_API_KEY") with
    | null -> failwith "API key not found. Set it in your environment variables."
    | key -> key

In [53]:
let saveJsonToFile (filePath: string) (jsonResponse: string) =
    try
        
        let data = JsonSerializer.Deserialize<obj>(jsonResponse)
        let options = JsonSerializerOptions(WriteIndented = true, Encoder = System.Text.Encodings.Web.JavaScriptEncoder.UnsafeRelaxedJsonEscaping)
        let jsonOutput = JsonSerializer.Serialize(data, options)

        File.WriteAllText(filePath, jsonOutput)
        printfn "JSON saved to %s" filePath
    with ex ->
        printfn "Failed to save pretty-printed JSON. Error: %s" ex.Message

In [54]:
let getAssetsByAddress (address: string) (filePath: string)=
    async {
        let apiKey = getApiKey()
        let pageNo = 1
        let baseUrl = "https://api.cardanoscan.io/api/v1/asset/list/byAddress"
        let url = $"{baseUrl}?address={address}&pageNo={pageNo}"

        use client = new HttpClient()
        client.DefaultRequestHeaders.Add("apiKey", apiKey)

        try
            let! response = client.GetAsync(url) |> Async.AwaitTask
            if response.IsSuccessStatusCode then
                let! content = response.Content.ReadAsStringAsync() |> Async.AwaitTask
                saveJsonToFile filePath content
                return Ok ()
            else
                let! errorContent = response.Content.ReadAsStringAsync() |> Async.AwaitTask
                printfn "Error Response Body: %s" errorContent
                return Error $"Error: {response.StatusCode} - {response.ReasonPhrase}"
        with ex ->
            printfn "Exception: %s" ex.Message
            return Error $"Exception: {ex.Message}"
    }

## Enter the Address and Call the Functions

You can replace the address in the cell below with the desired Cardano address to search for assets or information related to it.


In [56]:
let address = "01a2372d4f613b6d64da77f6047d4756b261b31619a54ec32e49b09797cfa1b556bfa358a865b6982a6e3a8c5f2e7144a430b1c7d9173c2ea4" 

In [57]:
let filePath = "./data/collectionassets.json"

async {
    let! result = getAssetsByAddress address filePath
    match result with
    | Ok () -> printfn "API response saved successfully."
    | Error errMsg -> printfn "Error: %s" errMsg
}
|> Async.RunSynchronously

JSON saved to ./data/collectionassets.json
API response saved successfully.
